In [1]:
import hoomd
import numpy as np
# import ex_render

from monk import mimse, pair, prep

In [3]:
seed = 1000
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(cpu, seed=seed)
N = 512
rng = prep.init_rng(seed + 1)
L = prep.len_from_phi(N, 1.2, dim=3)
snap = prep.approx_euclidean_snapshot(N, L, rng, dim=3, ratios=[80,20], diams=[1.0, 0.88])

sim.create_state_from_snapshot(snap)
# sim.create_state_from_gsd()

tree = hoomd.md.nlist.Tree(0.2)

integrator = hoomd.md.Integrator(dt=0.0025)
lang = hoomd.md.methods.Langevin(hoomd.filter.All(), 1.5)
lj = pair.KA_LJ(tree)
integrator.forces = [lj]
integrator.methods = [lang]
sim.operations.integrator = integrator

sim.run(0)
sim.state.thermalize_particle_momenta(hoomd.filter.All(), 1.5)

sim.run(1000)

sim.operations.integrator = None

integrator.forces.pop()
del integrator, lang

In [4]:
hoomd.write.GSD.write(sim.state, "test-3d.gsd")

In [5]:
fire = hoomd.md.minimize.FIRE(0.0025,
                              force_tol=1e-2,
                              angmom_tol=1e-2,
                              energy_tol=1e-7)

mimse_force = mimse.MIMSEForce(sim.state.box)
nve = hoomd.md.methods.NVE(hoomd.filter.All())
fire.forces = [lj]
fire.methods = [nve]

sim.operations.integrator = fire

In [6]:
sim.operations.writers.clear()

In [7]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(
    filter=hoomd.filter.All())
sim.operations.computes.append(thermodynamic_properties)

logger = hoomd.logging.Logger(categories=["scalar"])

logger.add(thermodynamic_properties, ["potential_energy"])

writer = hoomd.write.GSD(hoomd.trigger.Periodic(40), filename='test_mimse-3d.gsd', mode="wb", log=logger)

sim.operations.writers.append(writer)

table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=40),
                          logger=logger)

# sim.operations.writers.append(table)

In [8]:
sim.run(8000)

In [9]:
fire.forces.append(mimse_force)

In [10]:
for i in range(50):
    fire.reset()
    while not fire.converged:
        sim.run(40)
    print(lj.energy, mimse_force.energy)
    snap = sim.state.get_snapshot()
    mimse_force.add_bias(20.0, 1.0, snap.particles.position, np.zeros_like(snap.particles.position))

-3218.4252835892553 0.0
-3222.7472907210326 0.0
-3226.311043665231 0.0
-3230.7449022810333 0.0
-3230.062790207801 0.006316384671052333
-3229.679878458587 0.00830142092162589
-3230.044357233974 0.0010358396878018539
-3230.3800791515855 0.00012460668081359927
-3230.4602924346877 0.0008556192359066017
-3230.356663748018 0.0012177991277440015
-3230.4261656366257 0.0007508572861780749
-3229.927830046856 0.009066767171788115
-3232.0847012263403 0.0
-3231.617316658178 0.00426336173859454
-3231.614483334088 0.000775524688947371
-3230.8452462496134 0.011145935590732847
-3231.4687337885043 0.0024381199012424304
-3231.635895550579 0.004013340836266366
-3231.3097914878063 0.0026867185517431835
-3232.073169818242 0.0012644171484055278
-3232.4441717090904 0.0
-3232.149486683872 0.0004289752553177095
-3231.924756969412 0.0001907955579127929
-3231.4870265474683 0.010393328423674575
-3231.6631625570303 0.007214998858913191
-3231.3404756516543 0.013472941279601969
-3232.1571293396364 0.00415805643484645

In [9]:
mimse_force.nn

[11, 12, 13]

In [17]:
lj.energy

-793.7219270695075

In [15]:
len(mimse_force.nn)

50

In [10]:
sim.run(0)

In [54]:
x = hoomd._hoomd.make_scalar3(1,10,0)
im = hoomd._hoomd.make_int3(0,0,1)
flags = hoomd._hoomd.make_char3(0,0,0)
y = sim.state.box._cpp_obj.wrap(x, im, flags)